In [1]:
import pandas as pd
from datapreprocessfunction_code import function_process, readfile, rename_cols,annon,outers_join,add_column,label_paths
import seaborn as sns
from matplotlib import pyplot as plt
import os
import numpy as np
from plots_functions import plot3d
from collections import OrderedDict
import cmath
from sklearn.decomposition import PCA

<Figure size 576x720 with 0 Axes>

In [2]:
%store -r arr_great
%store -r PC_ref
%store -r common_de
%store -r lncRNAs_ref
arr_great = rename_cols(arr_great,"PC","gene_sym")

In [50]:
lncRNAs_ref.gene_sym.drop_duplicates().shape

(16825,)

In [3]:
pathways    = '/home/alva/Documents/lncRNAs_project_2020/Analysis_2021/DNA_methy_tcga/meth/de/filtered_results/'

In [4]:
kegg        = readfile(pathways,"great_PC_commonDE5lncRNAs_PC_KEGG_hsa_functional_classification.tsv")
goBP        = readfile(pathways,"great_PC_commonDE5lncRNAs_PC_GO_BP_goa_human_functional_classification.tsv")
kegg_geneid = readfile(pathways,"kegg_filtered_entrenzid_pathways_ensgene")

In [26]:
kegg.head()

,Genes,Process~name,num_of_Genes,gene_group,percentage%,Pvalue,Benjamini and Hochberg (FDR),Hommel singlewise process,Bonferroni single-step process,Hommel singlewise process.1,Hochberg step-up process,Benjamini and Yekutieli
0,127; 130; 131; 1738; 2023; 218; 221; 2538; 259...,hsa00010~Glycolysis_/_Gluconeogenesis,25,67,37.313433,0.029251,0.146024,1.0,1.0,1.0,1.0,0.934806
1,1738; 3417; 3418; 5161; 5162; 6389; 8801;,hsa00020~Citrate_cycle_(TCA_cycle),7,30,23.333333,0.552494,0.735208,1.0,1.0,1.0,1.0,1.000000
2,132158; 22934; 2821; 5214; 6120; 64080; 6888; ...,hsa00030~Pentose_phosphate_pathway,9,30,30.000000,0.298568,0.519007,1.0,1.0,1.0,1.0,1.000000
3,231; 51181; 57016; 6120; 7358; 7365; 79799; 9365;,hsa00040~Pentose_and_glucuronate_interconversions,8,34,23.529412,0.538982,0.722921,1.0,1.0,1.0,1.0,1.000000
4,231; 2762; 29926; 3098; 3099; 5209; 5214; 5373...,hsa00051~Fructose_and_mannose_metabolism,12,33,36.363636,0.120911,0.319281,1.0,1.0,1.0,1.0,1.000000


In [5]:
kegg[["Process~name","gene_group","num_of_Genes"]]

,Process~name,gene_group,num_of_Genes
0,hsa00010~Glycolysis_/_Gluconeogenesis,67,25
1,hsa00020~Citrate_cycle_(TCA_cycle),30,7
2,hsa00030~Pentose_phosphate_pathway,30,9
3,hsa00040~Pentose_and_glucuronate_interconversions,34,8
4,hsa00051~Fructose_and_mannose_metabolism,33,12
...,...,...,...
333,hsa05412~Arrhythmogenic_right_ventricular_card...,77,24
334,hsa05414~Dilated_cardiomyopathy,96,26
335,hsa05415~Diabetic_cardiomyopathy,203,47
336,hsa05416~Viral_myocarditis,60,15


In [6]:
kegg_geneid["Genes"] = kegg_geneid['Genes'].astype(int)

In [7]:
kegg_filtered    = kegg.query('Pvalue<=0.05')

In [8]:
kf               = (kegg_filtered.assign(Genes = kegg_filtered['Genes'].str.split(';'))
                    .explode('Genes')
                    .reset_index(drop=True))
kf['Genes'].replace('', np.nan, inplace=True)
kf.dropna(subset = ['Genes'], inplace=True)
kf['Genes']      = kf['Genes'].apply(lambda x: x.strip())
kf["Genes"]      = kf['Genes'].astype(int)
#kf.to_csv(pathways + 'kegg_filtered_entrenzid_pathways', sep='\t',index=False)

In [9]:
kf                = kf[["Genes","Process~name","gene_group","num_of_Genes"]]
pathways_kegg_eng = pd.merge(kegg_geneid[["ensembl_gene_id","Genes"]],kf,on='Genes')
pathways_kegg_eng['Process~name'] = pathways_kegg_eng['Process~name'].apply(lambda x:x.split('~')[1])
pathways_kegg_eng = rename_cols(pathways_kegg_eng,"ensembl_gene_id","gene_id")
pathways_kegg_eng['Process~name'] = pathways_kegg_eng['Process~name'].apply(lambda x:x.split('_')[0])
pathways_kegg_eng_ann = annon(PC_ref[["gene_id","gene_sym"]],"gene_id",pathways_kegg_eng)

In [10]:
df1 = (pathways_kegg_eng_ann.groupby('Process~name')['gene_sym'].
        apply(lambda x: ', '.join(x.astype(str))).reset_index())
df1['num_of_lncRNAs'] = df1.gene_sym.str.count(',')+1

In [11]:
pathways_kegg_eng_ann_list= (pathways_kegg_eng_ann['gene_sym'].str.split(', ')
                    .groupby(pathways_kegg_eng_ann['Process~name'])
                    .agg(lambda x: ', '.join(set(y for z in x for y in z)))
                    .reset_index())
%store pathways_kegg_eng_ann_list

Stored 'pathways_kegg_eng_ann_list' (DataFrame)


In [12]:
arr_great_ann = annon(PC_ref[["gene_id","gene_sym"]],"gene_sym",arr_great)
pathways_kegg_eng_ann_great = pd.merge(pathways_kegg_eng_ann[["gene_id","gene_sym","Process~name","gene_group","num_of_Genes"]],
         arr_great_ann[["gene_id","lncRNA","Distance"]],on="gene_id")

In [13]:
pathways_kegg_eng_ann_great = rename_cols(pathways_kegg_eng_ann_great,"Process~name","Pathway")

In [15]:
pathways_kegg_eng_ann_great_fil = pathways_kegg_eng_ann_great.query('Pathway !="Human" and Pathway !="Circadian" and Pathway !="Glutamatergic"')
pathways_kegg_eng_ann_great_fil = rename_cols(pathways_kegg_eng_ann_great_fil,"lncRNA","gene_ids")
pathways_kegg_eng_ann_great_fil.drop('gene_id',axis=1,inplace=True)
pathways_kegg_eng_ann_great_fil = rename_cols(pathways_kegg_eng_ann_great_fil,"gene_ids","gene_id")
pathways_kegg_eng_ann_great_fil_lnc = annon(lncRNAs_ref[['gene_id','gene_sym']],"gene_id",pathways_kegg_eng_ann_great_fil[["Pathway","gene_id","gene_group","num_of_Genes"]])

/home/alva/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4446: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/alva/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [16]:
pathways_kegg_eng_ann_great_fil_lnc = annon(common_de,"gene_sym",pathways_kegg_eng_ann_great_fil_lnc[["Pathway","gene_sym","gene_group","num_of_Genes"]])

In [17]:
pathways_kegg_eng_ann_great_fil_lnc.gene_sym.drop_duplicates().shape

(1408,)

In [21]:
pathways_kegg_eng_ann_great_fil_lnc.Pathway_y.drop_duplicates()

0                          cAMP
5                     Metabolic
6                     Alzheimer
7                           Ras
8                      PI3K-Akt
                  ...          
222862                 TGF-beta
223041                    Basal
223043                   Breast
394652                 Maturity
409612    Aldosterone-regulated
Name: Pathway_y, Length: 69, dtype: object

In [18]:
pathways_kegg_eng_ann_great_fil_lnc.shape

(19621642, 21)

In [45]:
%store pathways_kegg_eng_ann_great_fil_lnc

Stored 'pathways_kegg_eng_ann_great_fil_lnc' (DataFrame)
